<a href="https://colab.research.google.com/github/tiagosilveiraa/tccbigdata/blob/main/TCC_PUC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação de Bibliotecas


In [80]:
!pip install unidecode

In [81]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
from unidecode import unidecode
import unicodedata

# 1 - Coleta de dados
Inicialmente vamos carregar o dataset do Censo com os dados da pesquisa. Optei por hospedar o dataset no github para garantir a disponibilidade do mesmo, além disto minimizei os cabeçalhos para uma melhor manipulação dos dados.

O Link original do dataset é: https://pcrj.maps.arcgis.com/sharing/rest/content/items/97d55e185d114698ac5472f6f1c43758/data

In [82]:
base_censo = pd.read_excel('https://github.com/tiagosilveiraa/tccbigdata/raw/main/Dados_Censo_PopRua_2020.xlsx', sheet_name=1)
base_censo = base_censo.rename(str.lower, axis='columns')

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Para enriquecer os nossos dados vamos montar outro dataset descrevendo as regiões do Rio de Janeiro (Zonas).

In [83]:
page = requests.get('https://pt.wikipedia.org/wiki/Lista_de_bairros_do_Rio_de_Janeiro_por_%C3%8Dndice_de_Desenvolvimento_Social')

In [84]:
soup = BeautifulSoup(page.content, 'html.parser', from_encoding="utf-8")
tabela = soup.find('table', class_='wikitable')
linhas = tabela.findChildren(['tr'])

In [85]:
const_zonas = ['NORTE', 'SUL', 'OESTE', 'CENTRAL']
zonas = []
bairros = []
for linha in linhas:
  links = linha.find_all('a')
  for link in links:
    txt_link = link.get_text().replace('Zona ', '').upper()
    if (txt_link in const_zonas):  
      zonas.append(txt_link)  
    else:
      #Alguns bairros estão com dois espaços a mais.
      bairros.append(txt_link.replace('  ', ' '))

In [86]:
base_zonas = pd.DataFrame(list(zip(zonas, bairros)), columns=["zona", 'bairro'])

# 2 - Tratamento de Dados
Nesta etapa iremos fazer o tratamento de dados e a junção entre os dados das regiões e dos bairros transformando tudo em um único dataset.



In [87]:
df_censo = pd.DataFrame(base_censo)
df_zonas = pd.DataFrame(base_zonas)

Removendo acentos e colocando letras em maiúsculo.

In [88]:
df_censo = df_censo.applymap(lambda s: unidecode(s.upper()) if type(s) == str else s)
df_zonas = df_zonas.applymap(lambda s: unidecode(s.upper()) if type(s) == str else s)

# 2.1 - Verificando se existem bairros no censo que não estão no dataframe de zonas

In [89]:
bairros_censo = pd.DataFrame(np.unique(df_censo['bairro']), columns=['bairro'])
diff = bairros_censo.merge(df_zonas['bairro'], on='bairro', how="outer", indicator=True).drop_duplicates(keep=False)
diff.loc[diff['_merge']=='left_only']

,bairro,_merge
66,LAPA,left_only
70,LINS DE VASCONCELOS,left_only
130,VILA KENNEDY,left_only
131,VILA KOSMOS,left_only


# 2.1.1 - Corrigindo diferenças de escritta do dataframe de zonas

In [90]:
df_zonas.loc[df_zonas['bairro'] == 'LINS DE VASCONCELLOS', 'bairro'] = 'LINS DE VASCONCELOS'
df_zonas.loc[df_zonas['bairro'] == 'VILA COSMOS', 'bairro'] = 'VILA KOSMOS'

# 2.1.2 - Incluindo dois bairros ausentes no dataset de zonas.

In [91]:
bairros_faltantes = [
    {
        'zona':'CENTRAL',
        'bairro':'LAPA'
    },
    {
        'zona':'NORTE',
        'bairro':'VILA KENNEDY'
    }
]
df_zonas = df_zonas.append(bairros_faltantes, ignore_index=True)

Checando se ainda existe algum bairro presente na lista do censo e que não consta na relação de bairros com zonas classificadas.

In [92]:
bairros_censo = pd.DataFrame(np.unique(df_censo['bairro']), columns=['bairro'])
diff = bairros_censo.merge(df_zonas['bairro'], on='bairro', how="outer", indicator=True).drop_duplicates(keep=False)
diff.loc[diff['_merge']=='left_only']

,bairro,_merge


# 2.1.3 - Unindo os dados dos dois datasets

In [93]:
for i, r in df_zonas.iterrows():
  df_censo.loc[df_censo['bairro'] == r['bairro'], 'zona'] = r['zona']

In [94]:
df_censo.loc[df_censo['zona'].isnull()]

,id,rua_acolhimento,local_da_coleta_de_dados,unidade_de_acolhimento_us,metodo,turno,data,bairro,ap,codigo_da_rp,rp,codigo_da_ra,ra,latitude,longitude,situacao_entrevista,motivo_situacao_impossivel,dormiu_na_rua_ultimos_7_dias,respondeu_ao_questionario,questionario_de_observacao,idade,faixa_etaria,classificacao_idade,faixa_etaria_observada,sexo,genero,cor_raca,deficiencia_caminhar_ou_degraus,deficiencia_enxergar,deficiencia_ouvir,deficiencia_mental,deficiencia_nao_possui,deficiencia_ns_nr,documento_certidao_de_nascimento,documento_cpf,documento_carteira_de_identidade,documento_carteira_de_trabalho,documento_titulo_de_eleitor,documento_passaporte,documento_red,...,outras_fontes_auxilio_doenca,outras_fontes_aht_aluguel_social,outras_fontes_outros,outras_fontes_nao_recebe,outras_fontes_ns/nr,gravidez,acompanhamento_pre_natal,problema_saude_diabetes,problema_saude_pressao_alta,problema_saude_hiv_aids,problema_saude_sifilis_ou_ists,problema_saude_asma_bronq_pneum,problema_saude_tuberculose,problema_saude_cancer_tumores,problema_saude_hepatite,problema_saude_mental_epilepsia,problema_saude_lepra_outras,problema_saude_infeccao_urinaria,problema_saude_ferim_frat_outros,qual_unidade_saude_procura,faz_uso_drogas,drogas_tabaco,drogas_alcool,drogas_maconha_haxixe,drogas_crack_similares,drogas_cocaina,drogas_inalan_cola_solven_tiner,frequencia_tabaco,frequencia_alcool,frequencia_maconha/haxixe,frequencia_crack/similares,frequencia_cocaina,frequencia_inal_cola_solv_tiner,motivo_droga,motivo_local_uso_droga,sabe_ler_escrever,frequentou_escola,escolaridade,necessidade_sair_situacao,zona


# 2.2 - Limpando registros desnecessários
Para este estudo só são necessários os registros onde a pesquisa pode ser feita, sendo assim, iremos fazer a limpeza dos demais.

In [101]:
df_clean = df_censo.loc[df_censo['situacao_entrevista'] == 'POSSIVEL']
df_clean = df_clean.reset_index(drop=True)

# 2.3 - Tratamento de registros inválidos

Buscando por colunas com registros inválidos (NaN)

In [102]:
df_clean.columns[df_clean.isna().any()].tolist()

['unidade_de_acolhimento_us',
 'data',
 'onde_estava_antes_acolhimento',
 'abrigos_apresentam_problemas']

In [103]:
df_clean['data'] = df_clean['data'].fillna('NAO INFORMADA')

In [104]:
np.unique(df_clean.loc[df_clean['unidade_de_acolhimento_us'].isna(), ['rua_acolhimento']])

array(['RUA'], dtype=object)

Nota-se que todos as linhas onde os campos 'onde_estava_antes_acolhimento' e 'unidade_de_acolhimento_us' não estão preenchido são nas linhas onde o morador ainda não foi acolhido, por isso iremos fazer o preenchimento com a frase "NAO ACOLHIDO" 

In [105]:
df_clean['onde_estava_antes_acolhimento'] = df_clean['onde_estava_antes_acolhimento'].fillna('NAO ACOLHIDO')
df_clean['unidade_de_acolhimento_us'] = df_clean['unidade_de_acolhimento_us'].fillna('NAO ACOLHIDO')
df_clean['unidade_de_acolhimento_us'] = df_clean['abrigos_apresentam_problemas'].fillna('NAO INFORMADO')

# 3 - Análise exploratória

Perguntas que buscaremos respostas:
